In [2]:
# See if we have GPU access
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-12-12 14:42:43.910662: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 14:42:43.938592: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 14:42:43.938735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
# Optionally set debugging
# tf.debugging.set_log_device_placement(True)
# tf.debugging.set_log_device_placement(False)

In [4]:
# Load the annotation data
from json import load
from pathlib import Path

p = (Path().resolve() / "../pose_data/annotations.json").resolve()
annotations = load(p.open())

In [5]:
# Load annotated images in the following categories
ALLOW_ALL = True
ALLOWED_CATEGORIES = ["bicycling", "running", "walking"]
#ALLOWED_JOINTS = [1, 4, 9, 12, 13] # r knee, l knee, head, r shoulder, l shoulder
ALLOWED_JOINTS= [9]


valid_imgs = []
for i in range(len(annotations["act"])):
    if ALLOW_ALL or annotations["act"][i]["cat_name"] in ALLOWED_CATEGORIES:
        valid_imgs.append(i)
print(f"Indexed {len(valid_imgs)} images with valid categories")

Indexed 24987 images with valid categories


In [6]:
# Parse it to only images with a single person visible
import numpy as np

annorect_imgs = []

#for i in range(len(valid_imgs)):
i = 0
while len(annorect_imgs) < 512:
    i_img = valid_imgs[i]
    annolist = annotations["annolist"][i_img]
    annorects = annolist["annorect"]
    if isinstance(annorects, dict): # Only one annorect (one person)
        if "annopoints" in annorects.keys():
            point = annorects["annopoints"]["point"]
            # Check that every joint we care about is annotated]
            all_present = True
            for j in ALLOWED_JOINTS:
                all_present &= j in [p["id"] for p in point]
            if all_present:
                joints = []
                for j in ALLOWED_JOINTS:
                    joints.append([(p["x"], p["y"]) for p in point if p["id"] == j][0])
                annorect_imgs.append({"id": i_img, "joints": np.array(joints)})
    i += 1

print(f"Indexed {len(annorect_imgs)} images with valid annotations")

Indexed 11398 images with valid annotations


In [7]:
# Load the images from disk
import cv2
from skimage.transform import resize
from tqdm.auto import tqdm
HEIGHT = 256
WIDTH = 256

images = []

for i in range(len(annorect_imgs)):
    if i % 100 == 0:
        print('.', end="")
#for i in tqdm(range(len(annorect_imgs))):
    image_name = annotations["annolist"][annorect_imgs[i]["id"]]["image"]["name"]
    real_path = (Path().resolve() / "../pose_data/images/" / image_name).resolve()
    img = cv2.imread(real_path.as_posix())
    if img is not None:
        height, width, depth = img.shape
        images.append(
                {
                    "id": annorect_imgs[i]["id"],
                    "image": resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (HEIGHT, WIDTH)),
                    "joints": [(x / width, y / height) for (x, y) in annorect_imgs[i]["joints"]]
                }
            )
print("Done")


..

KeyboardInterrupt: 

In [ ]:
# Show a image from our dataset with the annotations
import matplotlib.pyplot as plt

def show_image(img, points, prediction = None):
    fig, ax = plt.subplots()
    ax.imshow(img)

    height, width, depth = img.shape

    x = [i[0] * width for i in points]
    y = [i[1] * height for i in points]
    plt.plot(x, y, "go")
    if prediction is not None:
        x_pred = [i[0] * width for i in prediction]
        y_pred = [i[1] * height for i in prediction]
        plt.plot(x_pred, y_pred, "ro")
    plt.show()
    
idx = np.random.randint(0, len(images))
show_image(images[idx]["image"], images[idx]["joints"])

In [ ]:
from tensorflow import keras
from keras import layers

REG_STRENGTH = 0.01

inputs = layers.Input(shape=(HEIGHT, WIDTH, 3))

head = layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", kernel_regularizer=keras.regularizers.L2(REG_STRENGTH), bias_regularizer=keras.regularizers.L2(REG_STRENGTH))(inputs)
head = layers.MaxPool2D(pool_size=2, strides=2)(head)
head = layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", kernel_regularizer=keras.regularizers.L2(REG_STRENGTH), bias_regularizer=keras.regularizers.L2(REG_STRENGTH))(head)
head = layers.MaxPool2D(pool_size=2, strides=2)(head)
head = layers.Conv2D(filters=32, kernel_size=2, padding="same", activation="relu", kernel_regularizer=keras.regularizers.L2(REG_STRENGTH), bias_regularizer=keras.regularizers.L2(REG_STRENGTH))(head)
head = layers.MaxPool2D(pool_size=2, strides=2)(head)
head = layers.GlobalMaxPool2D()(head)
head = layers.Dense(32, activation="relu", kernel_regularizer=keras.regularizers.L2(REG_STRENGTH), bias_regularizer=keras.regularizers.L2(REG_STRENGTH))(head)
head = layers.Dense(16, activation="relu", kernel_regularizer=keras.regularizers.L2(REG_STRENGTH), bias_regularizer=keras.regularizers.L2(REG_STRENGTH))(head)

outputs = layers.Dense(2, activation="sigmoid", kernel_regularizer=keras.regularizers.L2(REG_STRENGTH), bias_regularizer=keras.regularizers.L2(REG_STRENGTH))(head)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
# Split the test data into train/test splits

x = [i["image"] for i in images]
y = [i["joints"] for i in images]

TRAIN_SIZE = 0.75

# TODO: Randomize selection
idx = int(np.floor(TRAIN_SIZE * len(x)))
y_train = np.array(y[:idx])
y_test = np.array(y[idx:])
X_train = np.array(x[:idx])
X_test = np.array(x[idx:])
print("Done")

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Train the model
model.compile(optimizer="adam", loss=keras.losses.MeanSquaredError(), metrics=["acc"])
model.fit(X_train, y_train, batch_size=64, epochs=1, verbose=2)

In [ ]:
# Test
test_scores = model.evaluate(X_test, y_test)
print(f"Test loss: {test_scores[0]}")
print(f"Test accuracy: {test_scores[1]}")

In [ ]:
# Visualize a test point
idx = np.random.randint(0, len(X_test))
img = X_test[idx]
prediction = model(np.expand_dims(img, axis=0))
print(y_test[idx])
print(prediction)
show_image(img, y_test[idx], prediction)